In [241]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import accuracy_score
np.set_printoptions(threshold=sys.maxsize)

In [175]:
##get data from train_1.txt
d_1 = pd.read_csv("Train1.txt", sep= " ", header = None)
d_1 = d_1.drop(columns= [1])
d_1.columns = ["Col1", "Col2"]
d_1["Label"] = 0

d_1

,Col1,Col2,Label
0,-0.587726,5.105920,0
1,7.924885,3.145952,0
2,-2.785314,2.263386,0
3,0.214545,4.544092,0
4,7.376237,4.178282,0
...,...,...,...
1995,1.362357,3.705856,0
1996,3.471042,2.427904,0
1997,1.056309,-0.823561,0
1998,7.151360,4.281221,0


In [176]:
##get data from train_1.txt
d_2 = pd.read_csv("Train2.txt", sep= " ", header = None)
d_2 = d_2.drop(columns= [1])
d_2.columns = ["Col1", "Col2"]
d_2["Label"] = 1

d_2

,Col1,Col2,Label
0,-0.767074,5.780345,1
1,-2.413153,6.855475,1
2,0.243960,1.574809,1
3,-3.211301,8.693008,1
4,0.018983,-0.004968,1
...,...,...,...
1995,2.111239,4.998083,1
1996,-3.669095,5.680492,1
1997,6.686875,4.943677,1
1998,4.964905,2.231543,1


In [177]:
##get data from test_1.txt
t_1 = pd.read_csv("Test1.txt", sep= " ", header = None)
t_1 = t_1.drop(columns= [1])
t_1.columns = ["Col1", "Col2"]
t_1["Label"] = 0

t_1

,Col1,Col2,Label
0,1.191942,3.838177,0
1,5.414921,4.994841,0
2,-1.516117,3.832694,0
3,0.741797,6.544190,0
4,5.307803,0.968225,0
...,...,...,...
995,3.545965,2.530625,0
996,1.033023,2.646468,0
997,5.358833,2.523207,0
998,7.247577,4.610209,0


In [178]:
##get data from test_2.txt
t_2 = pd.read_csv("Test2.txt", sep= " ", header = None)
t_2 = t_2.drop(columns= [1])
t_2.columns = ["Col1", "Col2"]
t_2["Label"] = 1
t_2

,Col1,Col2,Label
0,4.523798,4.148166,1
1,1.842496,0.179926,1
2,-2.702575,6.720567,1
3,-1.776852,6.552999,1
4,0.300854,-0.313864,1
...,...,...,...
995,-3.233039,5.006300,1
996,1.570894,0.395078,1
997,-3.552151,5.620550,1
998,-1.507345,7.135042,1


In [179]:
#divide train data into train and validation set
train1 = d_1.loc[:1499]
valid1= d_1.loc[1500:]
train2 = d_2.loc[:1499]
valid2 = d_2.loc[1500:]
#print (train1.shape, train2.shape, valid1.shape, valid2.shape)

In [180]:
#merge train, validation and test data and reshuffle it.
tData = pd.concat([train1, train2])
tData = tData.sample(frac = 1)
tData.reset_index(drop = True, inplace = True)

vData = pd.concat([valid1, valid2])
vData = vData.sample(frac = 1)
vData.reset_index(drop = True, inplace = True)

train = pd.concat([t_1, t_2])
train = train.sample(frac = 1)
train.reset_index(drop = True, inplace = True)
train

,Col1,Col2,Label
0,2.084501,0.516846,0
1,4.619422,4.747131,0
2,-1.630989,-0.870099,1
3,-3.789630,5.029357,1
4,0.059931,2.331430,0
...,...,...,...
1995,0.284252,2.918311,0
1996,2.220296,4.808901,0
1997,2.459188,3.341694,1
1998,4.611763,4.642124,1


In [181]:
#seperate data and labels
td_data = tData.drop(columns= "Label")
td_label = np.array(tData["Label"])
td_label = np.reshape(td_label, (3000,1))

v_data = vData.drop(columns= "Label")
v_label = np.array(vData["Label"])
v_label = np.reshape(v_label, (1000,1))

tr_data = train.drop(columns= "Label")
tr_label = np.array(train["Label"])
tr_label = np.reshape(tr_label, (2000,1))

v_label.shape

(1000, 1)

In [292]:
#normalize the data
mu = np.mean(td_data, axis = 0)
sd = np.std(td_data, axis = 0)
print (mu, sd)
nor_td = (td_data - mu) / sd
nor_v = (v_data - mu) / sd
nor_tr = (tr_data - mu) / sd


Col1    1.48440
Col2    3.16029
dtype: float64 Col1    3.156768
Col2    2.387192
dtype: float64


In [277]:
def sig(x):
    return 1/(1+np.exp(-x))

def sig_der(x):
    return sig(x) *(1-sig (x))

def fowward_propogation(data, wh, wo):
    x = np.dot(data, wh)
    ax = sig(x)
    
    y = np.dot(ax, wo)
    ay = sig(y)
    return ax, ay, x, y

def mse_loss(ay, label):
    error = ((1 / 2) * (np.power((ay - label), 2)))
    return error.sum()
def plot_figs(t1, t2, t3, nodes):
    plt.figure(figsize=(10,7))
    plt.title("Number of nodes in hidden layer" + str(nodes))
    plt.plot(t1, color = 'red', label = 'Train')
    plt.plot(t3, color = 'green', label = 'Test')
    plt.plot(t2, color = 'yellow', label = 'Validation')
    plt.xlabel('Number of Epochs')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.show()
    
def accuracy(predicted, actual):
    return np.mean(actual == predicted)*100

In [295]:
def mlp(nor_td, td_label,nor_v, v_label,nor_tr, tr_label):
    input_node = 2
    hidden_node = [2, 4, 6, 8, 10, 13]
    output = 1
    train = defaultdict(list)
    valid = defaultdict(list)
    test = defaultdict(list)
    accur = defaultdict(list)
    
    for val in hidden_node:
        wh = np.random.uniform(size = (input_node, val))
        wo = np.random.uniform(size = (val, output))
        lr = 0.01
        
        for epoch in  range(1500):
            
            ax, ay, x, y = fowward_propogation(nor_td, wh, wo)
            
            #print(error, s)
            output_layer = ay - td_label
            oy_sig = sig_der(y) 
            oy_ax = ax
            
            oy_wo = np.dot(oy_ax.T, output_layer * oy_sig)
            
            oy_z = output_layer * oy_sig
            
            
            z_a = wo
            out_hid = np.dot(oy_z , z_a.T)
            out_hid_sig = sig_der(x) 
            feature = nor_td
            oy_wh = np.dot(feature.T, out_hid_sig * out_hid)
            wh -= lr * oy_wh
            wo -= lr * oy_wo
            ax1, ay1, x1, y1 = fowward_propogation(nor_td,wh, wo)
            ax2, ay2, x2, y2 = fowward_propogation(nor_v,wh, wo)
            ax3, ay3, x3, y3 = fowward_propogation(nor_tr,wh, wo)
            train[val].append(mse_loss(ay1, td_label))
            valid[val].append(mse_loss(ay2, v_label))
            test[val].append(mse_loss(ay3, tr_label))
            if (epoch>1000):
                if(valid[val][epoch] >= valid[val][epoch-1]):
                    print("breaking at epoch=" + str(epoch) + "with number of hidden nodes " + str(val))
                    break
                    
        ax3, ay3, x3, y3 = fowward_propogation(nor_tr,wh, wo)
#         print("wh" , wh)
#         print("wo" , wo)
        accur[val].append(accuracy(np.round(ay3), tr_label))
        
        

#     print(train)
#     print(valid)
#     print(test)
    for val in hidden_node:
        plot_figs(train[val], valid[val], test[val], val)
    
    print("########")
    print(accur)
    
  

In [296]:
mlp(nor_td, td_label,nor_v, v_label,nor_tr, tr_label )  

breaking at epoch=1002with number of hidden nodes 2
breaking at epoch=1002with number of hidden nodes 4
breaking at epoch=1001with number of hidden nodes 6
breaking at epoch=1001with number of hidden nodes 8
breaking at epoch=1001with number of hidden nodes 10
breaking at epoch=1002with number of hidden nodes 13
defaultdict(<class 'list'>, {2: [62.7], 4: [71.0], 6: [71.1], 8: [76.4], 10: [77.4], 13: [78.0]})
